# Tesla Stock Prediction with Prophet $TSLA 💰

In this project, we're looking at the stock adjusted closing prices of **Tesla, Inc** (`TSLA`) starting from 2010-06 to 2024-07. We're using data from a CSV file, so you can plug-in any stock data you want in the future. The data was gathered from [Yahoo Finance](https://finance.yahoo.com/quote/TSLA/history/?period1=1277818200&period2=1721865311&frequency=1wk).

In [36]:
#import libraries

import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

In [3]:
#import data

tsla = pd.read_csv('/Users/oluwa/Documents/DATA ANALYSIS/DATA PROJECTS/tsla.csv')

In [ ]:
# view data with head, shape, columns, describe, and dtypes

tsla.head()

In [ ]:
tsla.shape

In [ ]:
tsla.columns

In [ ]:
tsla.describe()

In [ ]:
tsla.dtypes

we want to change our 'Date' datatype from Object to datetime65[ns]

In [ ]:
#this will alter the DATE column and make it a DATETIME64ns datatype and overide the orignal loaded csv. 

tsla['Date'] = pd.to_datetime(tsla['Date'])
tsla.dtypes

Prepare our data for Prophet 

In [13]:
tsla['ds'] = pd.to_datetime(tsla['Date'])
tsla['y'] = tsla['Adj Close']  # or df['Close'] if you prefer

In [ ]:
# Select and display only 'ds' and 'y' columns
tsla2 = tsla[['ds', 'y']]
print(tsla2.head())  # Display the first few rows to verify

In [ ]:
tsla2

**Visualize** the data that we have. We don't want to start making predictions before looking at the past.

In [ ]:
#plot and shape Adj Close Price Over Time. 
tsla2.plot(x='ds', y='y', figsize=(15, 6), legend = False)
plt.xlabel('Date')
plt.ylabel('Adjusted Close Price')
plt.title('TSLA Adjusted Close Price Over Time')
plt.grid(True)
plt.show()

prepare data for modeling

In [ ]:
tsla2['ds'].dt.dayofweek 

In [ ]:
#view data by month
tsla2['ds'].dt.month

Creating/train model and fit(train) it into our dataframe

In [18]:
tslamodel = Prophet()

In [ ]:
tslamodel.fit(tsla2)

In [ ]:
#check indexes
tsla2[60:]

In [ ]:
tsla2[-60:]

Forcasting/predictions

In [ ]:
#create a forcast for your model
tslaforecast = tslamodel.predict(tsla2[['ds']])
tslaforecast.head()

In [ ]:
#plot forcast
tslamodel.plot(tslaforecast);

The black dots in the chart above is the real data points. 

The <span style='color: blue;'>dark blue line</span> is the predicted value - it's a line chart between the `yhat` points that we saw in the `forecast` DataFrame above.

The <span style='color: lightblue;'>light blue area</span> is the **confidence interval** - typically a 95% confidence window of the model prediction.

In [ ]:
ax = tslamodel.plot(tslaforecast).gca()
tsla2.plot(ax=ax, x='ds', y='y', label='actual', color='g');

In [ ]:
# find the diffrence between the data and our forcast
difference = tslaforecast['yhat'].values - tsla2['y'].values
difference

Create out of sample prections

We have set our `freq`uency to `MS` for month, or  `freq` option to `D`, for 'days'.

In [ ]:
# create a future dataframe for the next 180 days
tslafuture = tslamodel.make_future_dataframe(freq='D', periods=180)
tslafuture

In [ ]:
# OR we can also create a future dataframe for the next 3 Months
tslafuture = tslamodel.make_future_dataframe(freq='M', periods=3)
tslafuture

In [28]:
tslafutureforecast = tslamodel.predict(tslafuture)

In [ ]:
tslamodel.plot(tslafutureforecast);

plot trends

In [ ]:
tslamodel.plot_components(tslafutureforecast);

create interactive plots

In [28]:
import nbformat
print(nbformat.__version__)

5.10.4


In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(tslamodel, tslafutureforecast)

# model evaluation

Let's use the [Diagnostics](https://facebook.github.io/prophet/docs/diagnostics.html) library from `prophet` to validate our model using `cross_validation`. Run the cell below to import the library first:

In [30]:
from prophet.diagnostics import cross_validation

Now **create a DataFrame that is the result of running `cross_validation` on our model, with a horizon of 180, 90 or 60 days - your choice! 

The less days you choose, the longer it will take, and the more learning the model will do, because it will chop up your data into those blocks.

In [ ]:
tslavalidation = cross_validation(tslamodel, period='180 days', horizon='365 days')

In [ ]:
tslavalidation

# interpretation

Let's explore the chart above:
* Our horizontal (`x`) axis is how many days into the future we're predicting. As you can see, we tried to predict up to a maximum of 365 days (1 year). That's our **horizon**
* Our vertical (`y`) axis is the **Mean Absolute Error** - the average error in our predictions in stock price. We can see we are typically making errors of around 25-75.

**Our best case interpretation:**

We can see some clear highs and lows in our errors, which is not good. The main reason for that is that our data is `monthly` but we are doing our cross-validation `by day`. 


The **volatility** (waviness) or errors are fairly stable with some certainty. 

**Finally, let's visualize the errors** (differences) that between our model prediction and the seen reality. We will use the `mae` (Mean Absolute Error) as the metric, same as with our car sales predictions.

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(tslavalidation, metric='mae')

With the information you have, you can already start making decisions. The rest is up to how risk averse are you and what's your goal!


In [ ]:
tslamodel.plot(tslafutureforecast);

* We can see from the prediction `plot` that we have a point after which the model quickly starts to lose confidence.
* We can also see the same from the errors - as we try to predict further into the future, our accuracy goes down.
